In [ ]:
import numpy as np
import pandas as pd
import copy
import time
import random
import MetaTrader5 as mt5
import pandas_ta as ta
from tapy import Indicators
import math

In [ ]:
mt5.initialize()

In [ ]:
symbol='GOLD'

maxposition=0.1
pointer=str(random.randint(1000, 9999))

In [ ]:
# Get last 500 kandels 5 minutes for Symbol

def get_futures_klines(symbol, limit=1000):
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M5, 0, limit)
    df = pd.DataFrame(rates)
    df['Time'] = pd.to_datetime(df['time'], unit='s')
    df['Open']=df['open'].astype(float)
    df['High']=df['high'].astype(float)
    df['Low']=df['low'].astype(float)
    df['Close']=df['close'].astype(float)
    df['Volume']=df['tick_volume'].astype(float)
    df['rsi'] = round(ta.rsi(df['Close'], length = 14),3)
    df.drop(['open', 'high', 'low', 'close', 'tick_volume','spread','real_volume','time'], axis=1, inplace=True)
    return(df)

In [ ]:
def nadaraya_envelope(df, h=8, length=500, mult=3):
    lst = []
    src = df['Close'] # from dataframe
    y = []

    sum_e = 0
    for i in range(0, 500):
        sum_1 = 0
        sumw = 0

        for j in range(0, 500):
            w = math.pow(math.e,(-((i-j)**2)/(h*h*2)))
            sum_1 = sum_1 + src[j]*w
            sumw = sumw + w

        y2 = sum_1/sumw
        sum_e = sum_e + abs(src[i]-y2)
        y.append(y2)

    mae = sum_e/length*mult

    for i in range(1, 499):
        y2 = y[i]
        y1 = y[i-1]

        if src[i] > y1 + mae and src[i+1] < y1 + mae:
            lst.append([i,'s'])
        if src[i] < y1 - mae and src[i+1] > y1 - mae:
            lst.append([i,'b'])
    return lst

In [ ]:
# Open position for Sybol with 

def open_position(symbol, s_l, lot, comment="No specific comment", magic=0):
    
    mt5.initialize()
    prt('open: '+symbol+' quantity: '+str(lot))
    # Open a buy trade
    if (s_l=='long'):
        
        point = mt5.symbol_info(symbol).point 
        price = mt5.symbol_info_tick(symbol).ask 
        request = {
            "action": mt5.TRADE_ACTION_DEAL, 
            "symbol": symbol, 
            "volume": lot, 
            "type": mt5.ORDER_TYPE_BUY, 
            "price": price, 
            "sl": price - 300 * point,
            "tp": price + 300 * point, 
            "deviation": 10, 
            "magic": magic, 
            "comment": comment, 
            "type_time": mt5.ORDER_TIME_GTC, 
            "type_filling": mt5.ORDER_FILLING_IOC, # IOC (ETH) RETURN (XAU)
        }
        result = mt5.order_send(request)
    
    # Open a sell trade
    if (s_l=='short'):

        point = mt5.symbol_info(symbol).point 
        price = mt5.symbol_info_tick(symbol).bid 
        request = {
            "action": mt5.TRADE_ACTION_DEAL, 
            "symbol": symbol, 
            "volume": lot, 
            "type": mt5.ORDER_TYPE_SELL, 
            "price": price, 
            "sl": price + 300 * point,
            "tp": price - 300 * point, 
            "deviation": 10, 
            "magic": magic, 
            "comment": comment, 
            "type_time": mt5.ORDER_TIME_GTC, 
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        result = mt5.order_send(request)
    return result

In [ ]:
# Find all opened positions
def get_opened_positions():
    information = mt5.account_info()
    positions = mt5.positions_get()
    if len(positions)!=0:
        for position in positions:
            a = position.type
            entryprice = position.price_open
            profit = position.profit
            balance = information.balance
            id_position = position.ticket
            current_price = position.price_current
            volume = position.volume
            sl = position.sl
            if a==0:
                pos = 'long'
            elif a==1:
                pos = 'short'
            return ([pos,profit,balance,entryprice,id_position,current_price,sl,volume])
    else:
        pos = ''
        return ([pos])

In [ ]:
def check_and_close_orders(symbol):
    positions = mt5.positions_get(symbol=symbol)
    for position in positions:
        close_position(position)

In [ ]:
def get_symbol_price(symbol):
    price_1 = mt5.symbol_info_tick(symbol).bid
    price_2 = mt5.symbol_info_tick(symbol).ask
    return (price_1+price_2)/2

In [ ]:
# Changing stop loss
def change_stop_loss(symbol):
    
    position=get_opened_positions()    
        
    if position[0]=='long':
        price=mt5.symbol_info_tick(symbol).bid
        # Difference between current price and sl
        dif = position[5] - position[6]
        
        if position[6]<position[3]: # if sl less than entry price
            if position[3]+5<position[5]: # position[5] - current price
                # change stop loss
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": position[4],       
                    "symbol" : symbol,
                    "sl": position[3]+2,
                }
                result_SL = mt5.order_send(request)
                # close partially
                request1 = {
                    "action": mt5.TRADE_ACTION_DEAL, 
                    "symbol": symbol, 
                    'volume': position[7]/2,
                    "type": mt5.ORDER_TYPE_SELL, 
                    "position": position[4], 
                    "price": price, 
                    "deviation": 10, 
                    "magic": 0, 
                    "comment": "No comment!", 
                    "type_time": mt5.ORDER_TIME_GTC, 
                    "type_filling": mt5.ORDER_FILLING_IOC, 
                }
                result_pc = mt5.order_send(request1)
            
        else:    # if sl greater than entry price
            if dif>5:
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": position[4],
                    "symbol" : symbol,
                    "sl": position[6]+2,
                }
                result_SL = mt5.order_send(request)
                
                request1 = {
                    "action": mt5.TRADE_ACTION_DEAL, 
                    "symbol": symbol, 
                    'volume': position[7]/2,
                    "type": mt5.ORDER_TYPE_SELL, 
                    "position": position[4], 
                    "price": price,
                    "deviation": 10, 
                    "magic": 0, 
                    "comment": "No comment!", 
                    "type_time": mt5.ORDER_TIME_GTC, 
                    "type_filling": mt5.ORDER_FILLING_IOC, 
                }
                result_pc = mt5.order_send(request1)

    if position[0]=='short':
        price=mt5.symbol_info_tick(symbol).ask
        # Difference between current price and sl
        dif = position[5] - position[6]
        if position[6]>position[3]:
            if (position[3]-5)>position[5]:
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": position[4],
                    "symbol" : symbol,
                    "sl": position[3]-2,
                }
                result_SL = mt5.order_send(request)
                
                request1 = {
                    "action": mt5.TRADE_ACTION_DEAL, 
                    "symbol": symbol, 
                    'volume': position[7]/2,
                    "type": mt5.ORDER_TYPE_BUY, 
                    "position": position[4], 
                    "price": price,
                    "deviation": 10, 
                    "magic": 0, 
                    "comment": "No comment!", 
                    "type_time": mt5.ORDER_TIME_GTC, 
                    "type_filling": mt5.ORDER_FILLING_IOC, 
                }
                result_pc = mt5.order_send(request1)
        else:
            if dif<-5:
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": position[4],
                    "symbol" : symbol,
                    "sl": position[6]-2,
                }
                result_SL = mt5.order_send(request)

                request1 = {
                    "action": mt5.TRADE_ACTION_DEAL, 
                    "symbol": symbol, 
                    'volume': position[7]/2,
                    "type": mt5.ORDER_TYPE_BUY, 
                    "position": position[4], 
                    "price": price,
                    "deviation": 10, 
                    "magic": 0, 
                    "comment": "No comment!", 
                    "type_time": mt5.ORDER_TIME_GTC, 
                    "type_filling": mt5.ORDER_FILLING_IOC, 
                }
                result_pc = mt5.order_send(request1)
#     return result_SL, result_pc

In [ ]:
def check_if_signal(symbol):
    df = get_futures_klines(symbol)
    nad = nadaraya_envelope(df)
    signal="" # return value
    
    if nad[-1][0]==497 or nad[-1][0]==498:
        if nad[-1][1]=='b':
            if df['rsi'][nad[-1][0]]<30:
                signal='long'

        if nad[-1][1]=='s':
            if df['rsi'][nad[-1][0]]>70:
                signal='short'

    return signal

In [ ]:
def prt(message):
    # telegram message
    print(pointer+': '+message)

In [ ]:
def main(step):

    try:
        position=get_opened_positions()
        open_sl=position[0]
        if open_sl=="": # no position
            prt('Нет открытых позиций')
            # close all stop loss orders
            check_and_close_orders(symbol)
            signal=check_if_signal(symbol)

            if signal=='long':
                open_position(symbol,'long',maxposition)

            elif signal=='short':
                open_position(symbol,'short',maxposition)
        else:

            quantity=position[1]
            prt('Найдена открытая позиция '+open_sl)
            prt('Кол-во: '+str(quantity))
      
    except :
        prt('\n\nSomething went wrong. Continuing...')

In [ ]:
starttime=time.time()
timeout = time.time() + 60*60*16  # 60 seconds times 60 meaning the script will run for 12 hr
counterr=1

while time.time() <= timeout:
    try:
        prt("script continue running at "+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        main(counterr)
#         change_stop_loss(symbol)
        counterr=counterr+1
        if counterr>5:
            counterr=1
        time.sleep(20 - ((time.time() - starttime) % 20.0)) # 1 minute interval between each new execution
    except KeyboardInterrupt:
        print('\n\KeyboardInterrupt. Stopping.')
        exit()